In [2]:
import pandas as pd
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline

In [11]:
import pandas as pd
# Replace 'your_file.csv' with the path to your CSV file
df = pd.read_csv('Train.csv')
start_value = 1  # Start value for the auto-incrementing sequence
df.insert(0, 'Id', range(start_value, start_value + len(df)))
df.head()

,Id,eventID,hr,rawData,label
0,1,5635,75,426.0,1
1,2,5635,75,2267.0,1
2,3,5635,75,2002.0,1
3,4,5635,75,4090.0,1
4,5,5635,75,3057.0,1


In [12]:
# Function to replace negative values in a sequence
def replace_negative_values(sequence):
    non_negative_values = sequence[sequence >= 0]
    if len(non_negative_values) == 0:
        return sequence
    
    first_non_negative = non_negative_values.iloc[0]
    new_value = round(first_non_negative * 0.9) # Calculate 10% less and round to nearest whole number
    
    return sequence.replace(-1, new_value)

# Apply the function to the 'hr' column
df['hr'] = df.groupby('eventID')['hr'].transform(replace_negative_values)

# Display the modified DataFrame
print(df.head())
# Save the DataFrame to a new CSV file
df.to_csv('hr_dataset.csv', index=False)

   Id  eventID  hr  rawData  label
0   1     5635  75    426.0      1
1   2     5635  75   2267.0      1
2   3     5635  75   2002.0      1
3   4     5635  75   4090.0      1
4   5     5635  75   3057.0      1


In [7]:
# Function to replace -1 values with 10% below the next non-negative value
def replace_minus_one(df):
    for index, row in df.iterrows():
        if row['hr'] == -1:
            # Find the next non-negative value
            next_index = index + 1
            while next_index < len(df) and df.at[next_index, 'hr'] == -1:
                next_index += 1
            if next_index < len(df):
                next_value = df.at[next_index, 'hr']
                # Calculate 10% below the next non-negative value
                replacement_value = next_value * 0.9
                # Replace all -1 values in the sequence with the calculated value
                sequence_indices = range(index, next_index)
                df.loc[sequence_indices, 'hr'] = replacement_value
            else:
                # If there are no more non-negative values, interpolate the first value
                prev_index = index - 1
                while prev_index >= 0 and df.at[prev_index, 'hr'] == -1:
                    prev_index -= 1
                if prev_index >= 0:
                    prev_value = df.at[prev_index, 'hr']
                    # Interpolate the first value based on the previous non-negative value
                    df.at[index, 'hr'] = prev_value * 1.1  # 10% above the previous non-negative value
    return df

# Replace -1 values with 10% below the next non-negative value
df = replace_minus_one(df)

# Save the DataFrame to a new CSV file
df.to_csv('updated_dataset.csv', index=False)

In [13]:
# Replace 'your_file.csv' with the path to your CSV file
df = pd.read_csv('hr_dataset.csv')
print(df.head())

# Assume df is your DataFrame containing the 'hr' column that needs interpolation

# Define a function to perform cubic spline interpolation within each group
def interpolate_hr(group):
    # Extract the hr values and indices within the group
    hr_values = group['hr'].values
    indices = group.index.values
    
    # Perform cubic spline interpolation
    spline = CubicSpline([indices[0], indices[-1]], [hr_values[0], hr_values[-1]])
    interpolated_hr = spline(indices)
    
    # Update the 'hr' column in the group with interpolated values
    group['hr'] = interpolated_hr
    
    return group

# Apply the interpolation function to each group defined by 'eventID'
df_interpolated = df.groupby('eventID').apply(interpolate_hr)

print(df_interpolated.tail())
# Save the corrected DataFrame to a new CSV file
print("Saving the corrected dataset...")
df_interpolated.to_csv("corrected_dataset.csv", index=False, float_format='%.4f')

   Id  eventID  hr  rawData  label
0   1     5635  75    426.0      1
1   2     5635  75   2267.0      1
2   3     5635  75   2002.0      1
3   4     5635  75   4090.0      1
4   5     5635  75   3057.0      1
            Id  eventID   hr      rawData  label
493620  493621    45564  0.0   995.387993      2
493621  493622    45564  0.0  1008.981093      2
493622  493623    45564  0.0  1000.225913      2
493623  493624    45564  0.0   988.857927      2
493624  493625    45564  0.0   972.003617      2
Saving the corrected dataset...


In [8]:
import pandas as pd
# Replace 'your_file.csv' with the path to your CSV file
df = pd.read_csv('new_dataset.csv')
df.head()

,Id,eventID,hr,rawData,label
0,1,5635,75.0000,426.0,1
1,2,5635,75.0003,2267.0,1
2,3,5635,75.0006,2002.0,1
3,4,5635,75.0009,4090.0,1
4,5,5635,75.0012,3057.0,1


In [9]:
import pandas as pd
import numpy as np

# Assuming your DataFrame is named df
# Assuming your sequences are in a column named 'sequence'
# Assuming your eventIDs are in a column named 'eventID'
# Assuming your outcome classes are in a column named 'outcome'

# Function to apply oversampling to each group and maintain sequence integrity
def balance_sequences(df):
    balanced_dfs = []  # List to store balanced dataframes for each group

    for _, group_df in df.groupby('eventID'):
        # Get the count of each class in the group
        class_counts = group_df['label'].value_counts()

        # Find the maximum class count
        max_count = class_counts.max()

        # Resample each class to match the maximum count
        resampled_dfs = []
        for class_label, count in class_counts.items():
            if count < max_count:
                # Oversample the minority class to match the majority class
                resampled_df = group_df[group_df['label'] == class_label].sample(n=max_count - count, replace=True, random_state=42)
                resampled_dfs.append(resampled_df)

        # Concatenate the original and resampled dataframes
        if resampled_dfs:
            resampled_group_df = pd.concat([group_df] + resampled_dfs)
        else:
            resampled_group_df = group_df

        # Shuffle the dataframe to mix the samples
        resampled_group_df = resampled_group_df.sample(frac=1, random_state=42)

        # Append balanced group dataframe to list
        balanced_dfs.append(resampled_group_df)

    # Concatenate balanced group dataframes into a single dataframe
    balanced_df = pd.concat(balanced_dfs, ignore_index=True)

    return balanced_df

# Apply balancing while maintaining sequence integrity
balanced_df = balance_sequences(df)

# Now, balanced_df contains the balanced dataframe with sequences intact.


In [10]:
balanced_df.head()
sorted_df = balanced_df.sort_values(by='Id')
print(sorted_df.head(1300))
sorted_df.to_csv('new_balanced_dataset.csv', index=False)



         Id  eventID       hr  rawData  label
88545     1     5635  75.0000    426.0      1
91278     2     5635  75.0003   2267.0      1
90099     3     5635  75.0006   2002.0      1
90595     4     5635  75.0009   4090.0      1
91064     5     5635  75.0012   3057.0      1
...     ...      ...      ...      ...    ...
88354  1296     5635  75.3986   1806.0      1
90941  1297     5635  75.3989   1738.0      1
91447  1298     5635  75.3992   1784.0      1
89377  1299     5635  75.3995   1730.0      1
89587  1300     5635  75.3998   1742.0      1

[1300 rows x 5 columns]
